<a href="https://colab.research.google.com/github/momchilgeorgiev/2023DL/blob/main/Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, Input
from tensorflow.keras.models import Sequential, Model

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
(ds_train, ds_test), ds_info = tfds.load('mnist', split = ['train', 'test'], shuffle_files=True,
                             as_supervised = True, with_info = True)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [6]:
def norm(image, label):
  # cast image from int to float32 and normalize it
  # return the image and the label
  return tf.cast(image, tf.float32) / 255., label

In [7]:
# Apply the transformation on the whole ds. Use cached values
ds_train = ds_train.map(
    norm, num_parallel_calls = tf.data.AUTOTUNE
)

In [8]:
ds_train = ds_train.cache() # cache ds into memory
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128) # set batch the ds into size 128
ds_train = ds_train.prefetch(tf.data.AUTOTUNE) # get ds from cache

# Same protocol for ds_test
ds_test = ds_test.map(
    norm, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [18]:
CLASS_NUM = 10

In [19]:
model = tf.keras.models.Sequential([
  # INPUT
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  # HIDDEN LAYERS
  tf.keras.layers.Dense(128, activation='relu'),
  # OUTPUT
  tf.keras.layers.Dense(CLASS_NUM)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6
469/469 [==============================] - 2s 3ms/step - loss: 0.3514 - sparse_categorical_accuracy: 0.9034 - val_loss: 0.1941 - val_sparse_categorical_accuracy: 0.9448
Epoch 2/6
469/469 [==============================] - 2s 3ms/step - loss: 0.1623 - sparse_categorical_accuracy: 0.9538 - val_loss: 0.1300 - val_sparse_categorical_accuracy: 0.9616
Epoch 3/6
469/469 [==============================] - 2s 4ms/step - loss: 0.1162 - sparse_categorical_accuracy: 0.9666 - val_loss: 0.1104 - val_sparse_categorical_accuracy: 0.9661
Epoch 4/6
469/469 [==============================] - 2s 4ms/step - loss: 0.0894 - sparse_categorical_accuracy: 0.9744 - val_loss: 0.1007 - val_sparse_categorical_accuracy: 0.9698
Epoch 5/6
469/469 [==============================] - 2s 3ms/step - loss: 0.0716 - sparse_categorical_accuracy: 0.9791 - val_loss: 0.0893 - val_sparse_categorical_accuracy: 0.9716
Epoch 6/6
469/469 [==============================] - 2s 3ms/step - loss: 0.0588 - sparse_categorical_accu